In [1]:
!pip install torch
!pip install torchvision

In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [98]:
input_size = 784
hidden_size = 256
num_classes = 100
num_epochs = 10
batch_size = 100
lr = 3e-3

In [99]:
# Cargar el conjunto de datos de entrenamiento Fashion MNIST
train_data = dsets.FashionMNIST(root='./data', train=True,
                                transform=transforms.ToTensor(), download=True)

# Cargar el conjunto de datos de prueba Fashion MNIST
test_data = dsets.FashionMNIST(root='./data', train=False,
                               transform=transforms.ToTensor(), download = True)

In [100]:
train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size,
                                      shuffle = True)

In [101]:
class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.drop = nn.Dropout(0.2)
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.drop(out)
    out = self.fc2(out)
    return out

In [102]:
net = Net(input_size, hidden_size, num_classes)
# Verifica si tienes acceso a GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

Net(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (relu): ReLU()
  (drop): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=256, out_features=100, bias=True)
)

In [103]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [104]:

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_gen):
        # Cambia la forma de las imágenes y envíalas al dispositivo (GPU o CPU)
        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)

        optimizer.zero_grad()  # Pone a cero los gradientes
        outputs = net(images)  # Pasa las imágenes a través de la red
        loss = loss_function(outputs, labels)  # Calcula la pérdida
        loss.backward()  # Calcula los gradientes
        optimizer.step()  # Actualiza los parámetros

        # Imprime la pérdida cada 100 iteraciones
        if (i + 1) % 100 == 0:
            print('Epoca [%d/%d], Step [%d/%d], Loss: %.4f'
                  % (epoch + 1, num_epochs, i + 1, len(train_data) // batch_size, loss.item()))


Epoca [1/5], Step [100/600], Loss: 0.6463
Epoca [1/5], Step [200/600], Loss: 0.5043
Epoca [1/5], Step [300/600], Loss: 0.3684
Epoca [1/5], Step [400/600], Loss: 0.3987
Epoca [1/5], Step [500/600], Loss: 0.5360
Epoca [1/5], Step [600/600], Loss: 0.4387
Epoca [2/5], Step [100/600], Loss: 0.4468
Epoca [2/5], Step [200/600], Loss: 0.3719
Epoca [2/5], Step [300/600], Loss: 0.4068
Epoca [2/5], Step [400/600], Loss: 0.4119
Epoca [2/5], Step [500/600], Loss: 0.4265
Epoca [2/5], Step [600/600], Loss: 0.3073
Epoca [3/5], Step [100/600], Loss: 0.2552
Epoca [3/5], Step [200/600], Loss: 0.3278
Epoca [3/5], Step [300/600], Loss: 0.3442
Epoca [3/5], Step [400/600], Loss: 0.4582
Epoca [3/5], Step [500/600], Loss: 0.3764
Epoca [3/5], Step [600/600], Loss: 0.3144
Epoca [4/5], Step [100/600], Loss: 0.2855
Epoca [4/5], Step [200/600], Loss: 0.4554
Epoca [4/5], Step [300/600], Loss: 0.3796
Epoca [4/5], Step [400/600], Loss: 0.4725
Epoca [4/5], Step [500/600], Loss: 0.6359
Epoca [4/5], Step [600/600], Loss:

In [105]:
correct = 0
total = 0
for images, labels in test_gen:
    # Cambia la forma de las imágenes y envíalas al dispositivo (GPU o CPU)
    images = images.view(-1, 28*28).to(device)
    labels = labels.to(device)

    output = net(images)  # Pasa las imágenes a través de la red
    _, predicted = torch.max(output, 1)  # Obtiene las predicciones
    correct += (predicted == labels).sum().item()  # Suma las predicciones correctas
    total += labels.size(0)  # Suma el total de ejemplos

# Calcula y muestra la precisión
print('Accuracy: %.3f %%' % (100 * correct / total))

Accuracy: 85.600 %
